In [4]:
with open("./Desktop/blosum62.txt", "r") as file:
    BLOSUM62=[]
    dict_blosum = {}
    count = 0
    for line in file:
        count += 1
        if count == 1:
            row = line.rstrip("\n")[3:]
            dict_array = row.split("  ")
            for i in range(len(dict_array)):
                dict_blosum[dict_array[i]] = i
        else: 
            row = line.rstrip("\n")
            row = row[1:]
            array = []
            for num in row.split(" "):
                if num != "":
                    array.append(int(num))
            BLOSUM62.append(array)
    print(BLOSUM62)
    print(dict_blosum)

[[4, 0, -2, -1, -2, 0, -2, -1, -1, -1, -1, -2, -1, -1, -1, 1, 0, 0, -3, -2], [0, 9, -3, -4, -2, -3, -3, -1, -3, -1, -1, -3, -3, -3, -3, -1, -1, -1, -2, -2], [-2, -3, 6, 2, -3, -1, -1, -3, -1, -4, -3, 1, -1, 0, -2, 0, -1, -3, -4, -3], [-1, -4, 2, 5, -3, -2, 0, -3, 1, -3, -2, 0, -1, 2, 0, 0, -1, -2, -3, -2], [-2, -2, -3, -3, 6, -3, -1, 0, -3, 0, 0, -3, -4, -3, -3, -2, -2, -1, 1, 3], [0, -3, -1, -2, -3, 6, -2, -4, -2, -4, -3, 0, -2, -2, -2, 0, -2, -3, -2, -3], [-2, -3, -1, 0, -1, -2, 8, -3, -1, -3, -2, 1, -2, 0, 0, -1, -2, -3, -2, 2], [-1, -1, -3, -3, 0, -4, -3, 4, -3, 2, 1, -3, -3, -3, -3, -2, -1, 3, -3, -1], [-1, -3, -1, 1, -3, -2, -1, -3, 5, -2, -1, 0, -1, 1, 2, 0, -1, -2, -3, -2], [-1, -1, -4, -3, 0, -4, -3, 2, -2, 4, 2, -3, -3, -2, -2, -2, -1, 1, -2, -1], [-1, -1, -3, -2, 0, -3, -2, 1, -1, 2, 5, -2, -2, 0, -1, -1, -1, 1, -1, -1], [-2, -3, 1, 0, -3, 0, 1, -3, 0, -3, -2, 6, -2, 0, 0, 1, 0, -3, -4, -2], [-1, -3, -1, -1, -4, -2, -2, -3, -1, -3, -2, -2, 7, -1, -2, -1, -1, -2, -4, -3], [-1

In [3]:
def AlignmentWithAffineGapPenalities(v,w, BLOSUM62, dict_blosum, opening, extension):
    n = len(v)
    m = len(w)
    Middle=[]
    Lower = []
    Upper = []
    Backtrack = []
    Backtrackm=[]
    Backtrackl=[]
    Backtracku=[]
    for i in range(0, n+1):
        a=[]
        for j in range(0, m+1):
            a.append(0)
        Middle.append(a)
        Lower.append(a[:])
        Upper.append(a[:])
        Backtrackm.append(a[:])
        Backtrackl.append(a[:])
        Backtracku.append(a[:])
    for i in range(0, n+1):
        Upper[i][0] = -float("inf")
    for j in range(0, m+1):
        Lower[0][j] = -float("inf")
    for i in range(1, n+1):
        for j in range(1, m+1):
            Lower[i][j] = max(Lower[i-1][j]-extension, Middle[i-1][j]-opening)
            Upper[i][j] = max(Upper[i][j-1]-extension, Middle[i][j-1]-opening)
            indexi = dict_blosum[v[i-1]] 
            indexj = dict_blosum[w[j-1]]
            Middle[i][j] = max(Lower[i][j], Upper[i][j], Middle[i-1][j-1]+ BLOSUM62[indexi][indexj])

            if Lower[i][j] == Lower[i-1][j]-extension:
                Backtrackl[i][j] = 1
            elif Lower[i][j] == Middle[i-1][j]-opening:
                Backtrackl[i][j] = 2    
            if Middle[i][j] == Lower[i][j]:
                Backtrackm[i][j] = 1
            elif Middle[i][j] == Middle[i-1][j-1]+BLOSUM62[indexi][indexj]:
                Backtrackm[i][j] = 2
            elif Middle[i][j] == Upper[i][j]:
                Backtrackm[i][j] = 3
                
            if Upper[i][j] == Upper[i][j-1]-extension:
                Backtracku[i][j]  = 1
            elif Upper[i][j] == Middle[i][j-1]-opening:
                Backtracku[i][j]  = 2
    return (Backtracku, Backtrackm, Backtrackl, Middle[-1][-1])
            
    #lower
    #higher
    #middle

In [53]:
v="PRTEINS"
w="PRTWPSEIN"

In [4]:
Backtracku, Backtrackm, Backtrackl,score=AlignmentWithAffineGapPenalities(v,w, BLOSUM62,dict_blosum, 11, 1)
print(score)


161


In [5]:
def OutputPenaltyNoRecursion(Backtracku, Backtrackm, Backtrackl, v, w):
    string_1 = ""
    string_2 = ""
    i = len(v)
    j = len(w)
    lower = False
    upper = False
    middle = True
    while i != 0 and j !=0:
        if middle:
            if Backtrackm[i][j] == 1:
                #string_1 = peptide_1[i-1]+ string_1
                #string_2 = "-" + string_2
                lower = True
                middle = False
            elif Backtrackm[i][j] == 2:
                string_1 = v[i-1]+ string_1
                string_2 = w[j-1] + string_2
                #print(v[i-1], w[j-1])
                i = i-1
                j = j-1
                
            elif Backtrackm[i][j] == 3:
                upper = True
                middle = False
        if lower:
            if Backtrackl[i][j] == 1:
                string_1 = v[i-1] + string_1
                string_2 = "-" + string_2
                i = i-1
            elif Backtrackl[i][j] == 2:
                string_1 = v[i-1] + string_1
                string_2 = "-" + string_2
                i = i-1
                middle = True
                lower = False
        if upper:
            if Backtracku[i][j] == 1:
                string_1 = "-" + string_1
                string_2 = w[j-1] + string_2  
                j = j-1
            elif Backtracku[i][j] == 2:
                string_1 = "-" + string_1
                string_2 = w[j-1] + string_2  
                j = j-1
                middle = True
                upper = False
    print(string_1)
    print(string_2)
    return (string_1,string_2)

In [6]:
OutputPenaltyNoRecursion(Backtracku, Backtrackm, Backtrackl, v, w)

EAWLNYP---MYPKI-PYVCWWFANG---HQP------PG---------ASRRNIDKIGLHCWEQDMAGRQTLKDNPFWTTWCCLLPEQVPVKDDHTCPHDIPWLIIQHGVA
EAALNYPMNYMYRLMAPPREEQNANGMMRTQPCLQEVMPGQGFRFQKDTGSRPNIDWI-------DMAGRQTLK---FWTTWCCLLPTQVPV----TCDH---WLIIQHGHA


('EAWLNYP---MYPKI-PYVCWWFANG---HQP------PG---------ASRRNIDKIGLHCWEQDMAGRQTLKDNPFWTTWCCLLPEQVPVKDDHTCPHDIPWLIIQHGVA',
 'EAALNYPMNYMYRLMAPPREEQNANGMMRTQPCLQEVMPGQGFRFQKDTGSRPNIDWI-------DMAGRQTLK---FWTTWCCLLPTQVPV----TCDH---WLIIQHGHA')

In [61]:
v="YHFDVPDCWAHRYWVENPQAIAQMEQICFNWFPSMMMKQPHVFKVDHHMSCRWLPIRGKKCSSCCTRMRVRTVWE"
w="YHEDVAHEDAIAQMVNTFGFVWQICLNQFPSMMMKIYWIAVLSAHVADRKTWSKHMSCRWLPIISATCARMRVRTVWE"

In [1]:
v="EAWLNYPMYPKIPYVCWWFANGHQPPGASRRNIDKIGLHCWEQDMAGRQTLKDNPFWTTWCCLLPEQVPVKDDHTCPHDIPWLIIQHGVA"
w="EAALNYPMNYMYRLMAPPREEQNANGMMRTQPCLQEVMPGQGFRFQKDTGSRPNIDWIDMAGRQTLKFWTTWCCLLPTQVPVTCDHWLIIQHGHA"



In [37]:
def middleEdge(v,w, BLOSUM62, dict_blosum):
    n = len(v)
    m = len(w)
    middle = int(m/2)
    store = []
    for i in range(0, n+1):
        store.append(0)
    for i in range(1, n+1):
        store[i] = store[i-1] - 5
        
    
    for j in range(0, middle):
        array = []
        array.append(store[0] -5)
        for i in range(1, n+1):
            indexi = dict_blosum[v[i-1]] 
            indexj = dict_blosum[w[j]] 
            array.append(max(store[i]-5, store[i-1]+BLOSUM62[indexi][indexj], array[i-1]-5))
        store = array[:]
    fromSource = store[:]
    #print(fromSource)
    
    sink_store = []
    for i in range(0, n+1):
        sink_store.append(0)
    for i in range(1, n+1):
        sink_store[i] = store[i-1] -5
    
    backtrack =[]
    for j in range(0, m-middle):
        sink_array = []
        sink_array.append(sink_store[0]-5)
        backtrack.append(1)
        for i in range(1, n+1):
            indexi=dict_blosum[v[n-i]]
            indexj= dict_blosum[w[m-j-1]]
            sink_array.append(max(sink_store[i]-5, sink_store[i-1]+BLOSUM62[indexi][indexj], sink_array[i-1]-5))
            if j == m-middle-1:
                if sink_array[i] == sink_store[i-1]+BLOSUM62[indexi][indexj]:
                    backtrack.append(2)
                elif sink_array[i] == sink_array[i-1]-5:
                    backtrack.append(3)
                elif sink_array[i] == sink_store[i] -5:
                    backtrack.append(1)
                

        sink_store = sink_array[:]
   
    toSink = sink_store[::-1]
    backtrack = backtrack[::-1]
    #print(toSink)
    #print(backtrack)
  
    middle_array = [fromSource[i] + toSink[i] for i in range(len(fromSource))]
   
    max_value = max(middle_array)
    max_index = 0
    while middle_array[max_index] != max_value:
        max_index += 1
        
    #for i in range(len(middle_array)):
    #    if middle_array[i] == max_value:
    #        max_index = i
    #print(max_index, middle)
    #midEdge = "(" + str(max_index) +" ," + str(middle) + ")"
    
    #if backtrack[max_index] == 1:
        #print(max_index,middle+1)
        #midEdge += " " + "(" + str(max_index) +" ," + str(middle+1) + ")"
    #elif backtrack[max_index] == 2:
        #print(max_index+1, middle+1)
        #midEdge += " " + "(" + str(max_index+1) +" ," + str(middle+1) + ")"
    #elif backtrack[max_index] == 3:
        #print(max_index+1,middle)
        #midEdge += " " + "(" + str(max_index+1) +" ," + str(middle) + ")"
    
    return (max_value,max_index, backtrack[max_index] )
    
    
    

In [318]:
middleEdge(v,w, BLOSUM62, dict_blosum)

(297, 512, 2)

In [317]:
with open("./Desktop/test_46.txt") as file:
    count = 0
    for line in file:
        count +=1
        if count == 1:
            v = line.rstrip("\n")
        if count == 2:
            w = line.rstrip("\n")
            

In [482]:
def LinearSpaceAlignment(v,w,BLOSUM62, dict_blosum, result, top):
    n = len(v)
    m = len(w)
    if n == 0:
        #result.append(1)
        #print("n is 0", m)
        for i in range(m):
            result.append(1)
            #print(1)
            
        return 
    if m == 0:
        #result.append(3)
        #print("m is 0", n)
        for i in range(n):
            result.append(3)
           # print(3)
            result = [3] +result
        return 
    middle = int(m/2)
    #print("v",v)
   # print("w",w)
    score,midNode,midEdge = middleEdge(v,w,BLOSUM62, dict_blosum)
    if top:
        print(score)
    top_v = v[0:midNode]
    left_w = w[0:middle]
    LinearSpaceAlignment(top_v,left_w,BLOSUM62, dict_blosum, result, False)
    result.append(midEdge)
    #print(midEdge)
    if midEdge == 1 or midEdge == 2:
        middle = middle + 1
    if midEdge == 3 or midEdge == 2:
        midNode = midNode +1
    bottom_v = v[midNode:]
    right_w = w[middle:]
    LinearSpaceAlignment(bottom_v,right_w,BLOSUM62, dict_blosum, result, False)

In [380]:
v[9:]
        
    

'Y'

In [475]:
v="PLEASANTLY"
w="MEANLY"

In [404]:
v="PPPPPPP"
w="PPPPPPPPPPPPP"

In [479]:
result = []
LinearSpaceAlignment(v,w,BLOSUM62, dict_blosum, result, True)


1387


In [480]:
print(len(result))


4121


In [481]:
print(len(result))
width = 0
height = 0
for i in range(len(result)):
    
    if result[i] == 1 or result[i] ==2:
        width =1 +width
       
    if result[i] == 2 or result[i] == 3:
        height =1 + height
print(height)
print(width)

        

4121
3934
3800


In [438]:
print(len(v))
print(len(w))

    

4050
4031


In [449]:
def Reconstruct(v, w, result):
    string_1 = ""
    string_2 = ""
    index_v = 0
    index_w = 0
    for edge in result:
        if edge == 1:
            #print(edge, index_v, index_w)
            string_2 += w[index_w]
            index_w += 1
            string_1 += '-'
        elif edge == 2:
            #print(index_v)
           # print(edge, index_v, index_w)
            string_1 += v[index_v]
            string_2 += w[index_w]
            index_v += 1
            index_w += 1
        elif edge == 3:
            #print(edge, index_v, index_w)
            string_1 += v[index_v]
            index_v += 1
            string_2 += '-'
    
        
    print(len(result))
    print(len(string_1))
    print(len(string_2))
    print(string_1)
    print(string_2)
    return (string_1, string_2)

In [472]:
a,b=Reconstruct(v,w, result)

IndexError: string index out of range

In [465]:
count = 0
for i in range(len(a)):
    if a[i] == "-":
        count -= 5
    elif b[i] == "-":
        count -= 5
    else:
        index_a = dict_blosum[a[i]]
        index_b = dict_blosum[b[i]]
        count += BLOSUM62[index_a][index_b]
    
print(count)

16974


In [478]:
with open("./Desktop/test_48.txt") as file:
    count = 0
    for line in file:
        count +=1
        if count == 1:
            v = line.rstrip("\n")
        if count == 2:
            w = line.rstrip("\n")

In [467]:
v

'PTGQSYVTTARTTAECRVLHVMPFNYHMASIMDSYVFLNFGPALCMHEWYLCTMRCGWSKVGLGYMTCFCKNYHMSVKDAAYDGDKEMDGMTKWCVMPNCMWENEAQDQMQAWDSKGWQDFCDDIKAGMQFIWDSEPHGNFSEIMSMPFDIDVTIFHMQEPEIVQWTMNPQHSPHRPKSCTMASWRTQHHTAWNHCPVSASAFQPQVDVCDNVRFYGETAMNIVGGQAEAEKMKIHPSYQGHIHLCIGNEDTDGQQLWCQNHMQHEPFRYNDSDGDVTYQKHPACAAIPNIHSWFQPWGIDYQSNRQFGNQMDECYDLWALRVWDEPSVTWYYRHDLHDHSESWQRCETNVMWYKGAKDMRGDLWSPRVMIMVPFLTVWRCGVTCGWLWPKSFSKAMMRAQKIHEFPQQRIKTNGAKPDNEREWQAHHAFNTECKFVGPKPILLSKPWRQVDYDYCSFSDDMHFRKCVLTDEFFNVVSTKMVSQCWFWADTLNPEVSNQFMTQEYIVKMTSVCEVLNGVGGLPFVTADSCSSPVIEWGLWTNDQWEGFFKLYWVMLDNDKNPVKWPHNRGIVHGMWPIWWIEQNPIKVGQACMWYPLIDNYWEDNRDVLKPKEDMMAIDISGQVKGWATDIRPSSWSLYIIPDMVWRGSLCDLARVEYEHKPWHNCTTYHMRCVIFYYFAPIGNHNDATIPGWAEWCYWPKMWEGYVMVNCFTEQQHQAEAAVAWGWYGCTPNVPPVSPIMQSFKMFICPNQFQDLKLMQDPCWVLNKFSVNERQLDHCPMDASDHWSPSHNRWNLTFQAWPGRQEFAWPVLFFFSDVWWDAHDYIYVNVMGYTVYHAWSASWVVTQLGNIHGECWNCMVPPEIVMSNTNQKYEHYMIASREMVTPHRRRYAVCTFRNLAWKSFDQQFFCRENFIGIFPADCGIIKCEVFRDLQEFFDRENSKCDQNSQKNMHKFKYCFQFQPQDPVKQRLNPVHPWCRSEEDGLRTQEDIVRPAQYNE

In [290]:
with open("./Desktop/compare_47.txt") as file:
    count = 0
    for line in file:
        count +=1
        if count == 1:
            a = line.rstrip("\n")
        if count == 2:
            b = line.rstrip("\n")
    

In [356]:
with open("./Desktop/check_47.txt") as file:
    count = 0
    for line in file:
        count +=1
        if count == 1:
            a = line.rstrip("\n")
        if count == 2:
            b = line.rstrip("\n")
    

8230

In [359]:
len(b)

8230

In [361]:
count = 0
for i in range(len(a)):
    if a[i] == "-":
        count -= 5
    elif b[i] == "-":
        count -= 5
    else:
        index_a = dict_blosum[a[i]]
        index_b = dict_blosum[b[i]]
        count += BLOSUM62[index_a][index_b]
    
print(count)
    

17179


In [303]:
len(rarray)

4404

In [308]:
len(a)

4404

In [309]:
len(b)

4404

In [230]:
a="happy"
a[]

In [253]:
file = open("./Desktop/result_47.txt", "w")
string1, string2 = Reconstruct(v,w, result)
file.write(string1)
file.write("\n")
file.write(string2)
file.close()

WD--YI---LFYQVSFYPQVLSSNYYDDWCQLFEAYEIIIDNFCLMEECGLNYLLSFANLQDITNPH----Y-QHDRKMHASPNRSATHANTNDDRFFGMNSYC-H-YDALATEGLLVACMYCFSYELWEGYWNCL--IE---IMDGRMAIGQLCTGTYMKKAQKEVFHLRHPEASMEALHCHYVNDKQHKCDAMWSRPAHEGDVLYQVH--KI-CYL-HHQGR--C-----P-WEYWSFHVWADQ---C--SVLGFKVSVN-SWSTYIKQIIGNEPWITYASFHHAS-MCFHVVVPGKTGNEW-C---Q-KPMFYLTLVARHVRVNFISK-SHK--KKPYQAKFFD---CLESGCDGVMYHYEFKGMDCIPD-WNLYP----TNNQFRWR-DYAFWSYYGIWRIIQWSRFCT-HMA-D-S-E--NQKDLPCWPVPKHWITNWHQRDYAGSGRA-YPNYIYDYN------PTSSWFPCAARYENEPRQYF-QC-Q-T-CVYWSELLR-RHPWPVDWMSRLHPDNGVLWYTLNLLVCT-----HA-K--QRQPLEAW--GN-MR-VTHFVEAGRDTIGENPLAACCDDPKTTHICPVRQKWVCQLYQNPGLHWDCPYWHKMTCEFVKKLVADVWSSAGLIISNMTVYRRTPNSVIICKIGEMCMTRSGGPVEFYGHEHWLMSRWCL------G-REVKQSYDFIRCFPSRKWKTCQMS--NHTVDYQQQYTGWRSENPAAFLRFSRR-YYWKGFNECIHIAWNMAC-LYRF-TFEHADNDEPMFFNGWVLERTCQWFREKHCYDEVHTHKNWMEPEYTFRIPGIWYYHEYNNKWYCSVNHITKGPHQKIEPALTV-H-----EN--CVMTDQNVCYYQNNTWDTCPILYSKMEWHHENIEEVRCSRHCMHN-------NCE-EEYKVSQRIDDHICPACVDPGKNYGVKDEFTVAEWNIRQMG----Y--DG---Q----FEEPD-K------EHEF----

In [156]:
def findTopologicalOrdering(graph):
    Nodes = set()
    In = {}
    Out = {}
    dict_graph = {}
    for path in graph:
        snode = int(path.split(" -> ")[0])
        enodes = list(map(int, path.split(" -> ")[1].split(",")))    
        Nodes.add(snode)
        for enode in enodes:
            Nodes.add(enode)
        dict_graph[snode] = enodes
        Out[snode] = len(enodes)
        for enode in enodes:
            if enode in In.keys():
                In[enode] += 1
            else:
                In[enode] = 1
    for node in Nodes:
        if node not in Out.keys():
            Out[node] = 0
        if node not in In.keys():
            In[node] = 0
    results = []
    Visited = set()
    Unvisited = Nodes - Visited
    while len(Unvisited) > 0:
        for node in Unvisited:
            if In[node] == 0:
                results.append(node)
                Visited.add(node)
                Unvisited = Nodes - Visited
                if node in dict_graph.keys():
                    enodes = dict_graph[node]
                    for enode in enodes:
                        In[enode] -= 1
    return results
            
    
    

In [158]:
graph = []
with open("./Desktop/test_49.txt") as file:
    for line in file:
        graph.append(line.rstrip("\n"))
print(graph)

['0 -> 24,26,3,36,4', '1 -> 25,31,34,35', '11 -> 24,32', '12 -> 30,32,36', '13 -> 23,25', '14 -> 17,18', '16 -> 17,22,24,25', '17 -> 21', '18 -> 22', '19 -> 26', '2 -> 32', '20 -> 26,35', '22 -> 27,31', '23 -> 25', '24 -> 33,35', '25 -> 31', '26 -> 29', '27 -> 36', '28 -> 34', '3 -> 26,27,30,35,6', '30 -> 32,34', '31 -> 34', '32 -> 36', '33 -> 35', '4 -> 22,5,6', '5 -> 28,29', '6 -> 35', '7 -> 18', '8 -> 35', '9 -> 30,36']


In [136]:
graph=["1 -> 2", "2 -> 3", "4 -> 2", "5 -> 3"]

In [159]:
results = findTopologicalOrdering(graph)
for node in results:
    print(node, end=", ")

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 

In [51]:
1 in dic.keys()

True

In [100]:
a = {1:[1,2,3]}
a[1][0]

1

In [101]:
for i in range(0,len(a[1])):
    print(a[1][i])
    if a[1][i] == 1:
        

1
2
3


In [217]:
def MultipleAlignment(u, v, w):
    l = len(u)
    m = len(v)
    n = len(w)
    S = []
    Backtrack = []
    for i in range(l+1):
        b = []
        c= []
        for j in range(m+1):
            a = []
            for k in range(n+1):
                a.append(0)
            b.append(a[:])
            c.append(a[:])
        S.append(b[:])
        Backtrack.append(c[:])
    
    for i in range(1, l+1):
        for j in range(1, m+1):
            for k in range(1, n+1):
                if u[i-1] == v [j-1] and u[i-1] == w[k-1]:
                    S[i][j][k] = max(S[i-1][j][k], S[i][j-1][k], S[i][j][k-1], 
                                     S[i-1][j-1][k], S[i-1][j][k-1], S[i][j-1][k-1], S[i-1][j-1][k-1]+1)
                else:
                    S[i][j][k] = max(S[i-1][j][k], S[i][j-1][k], S[i][j][k-1], 
                                    S[i-1][j-1][k], S[i-1][j][k-1], S[i][j-1][k-1],S[i-1][j-1][k-1])
                #print(S[i][j][k])
                if S[i][j][k] == S[i-1][j][k]:
                    Backtrack[i][j][k] = 0
                elif S[i][j][k] == S[i][j-1][k]:
                    Backtrack[i][j][k] = 1
                elif S[i][j][k] == S[i][j][k-1]:
                    Backtrack[i][j][k] = 2
                elif S[i][j][k] == S[i-1][j-1][k]:
                    Backtrack[i][j][k] = 3
                elif S[i][j][k] == S[i-1][j][k-1]:
                    Backtrack[i][j][k] = 4
                elif S[i][j][k] == S[i][j-1][k-1]:
                    Backtrack[i][j][k] = 5
                elif S[i][j][k] == S[i-1][j-1][k-1]+1:
                    Backtrack[i][j][k] = 6
                elif S[i][j][k] == S[i-1][j-1][k-1]:
                    Backtrack[i][j][k] = 7
                    
    

                
    print(S)
    print(Backtrack)
    return (S[-1][-1][-1],Backtrack)
        
                

In [225]:
def BacktrackMultiple(Backtrack, u,v,w):
    string_1 = ""
    string_2 = ""
    string_3 = ""
    i = len(u)
    j = len(v)
    k = len(w)
    while i != 0 and j !=0 and k != 0:
        print(i,j,k)
        if Backtrack[i][j][k] == 0:
            string_1 = u[i-1] + string_1
            string_2 = "-" + string_2
            string_3 = "-" + string_3
            i = i-1
        elif Backtrack[i][j][k] == 1:
            string_2 = v[j-1] + string_2
            string_1 = "-" + string_1
            string_3 = "-" + string_3
            j = j-1
        elif Backtrack[i][j][k] == 2: 
            string_3 = w[k-1] + string_3
            string_1 = "-" + string_1
            string_2 = "-" + string_2
            k = k-1
        elif Backtrack[i][j][k] == 3:
            string_1 = u[i-1] + string_1
            string_2 = v[j-1] + string_2
            string_3 = "-" + string_3
            i = i-1
            j = j-1  
        elif Backtrack[i][j][k] == 4: 
            string_1 = u[i-1] + string_1
            string_3 = w[k-1] + string_3
            string_2 = "-" + string_2
            i = i-1
            k = k-1
        elif Backtrack[i][j][k] == 5:
            string_2 = v[j-1] + string_2
            string_3 = w[k-1] + string_3
            string_1 = "-" + string_1
            j = j-1
            k = k-1
        elif Backtrack[i][j][k] == 6 or Backtrack[i][j][k] == 7:
            string_1 = u[i-1] + string_1
            string_2 = v[j-1] + string_2
            string_3 = w[k-1] + string_3
            i = i-1
            j = j-1
            k = k-1
    while i != 0:
        string_1 = u[i-1] + string_1
        string_2 = "-" + string_2
        string_3 = "-" + string_3   
        i = i-1
    while j != 0:
        string_1 = "-" + string_1
        string_2 = v[j-1] + string_2
        string_3 = "-" + string_3
        j= j-1
    while k != 0:
        string_1 = "-" + string_1
        string_2 = "-" + string_2
        string_3 = w[k-1] + string_3
        k= k-1
    print(string_1)
    print(string_2)
    print(string_3)

In [185]:
u = "ATATCCG"
v = "TCCGA"
w = "ATGTACTG"


In [215]:
score,backtrack=MultipleAlignment(u,v,w)
score

[[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 1, 1, 1, 1, 1, 1]], [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 1, 1]], [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 2, 2, 2, 2]], [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 2, 2, 2, 2]], [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 0, 1, 1, 1, 1, 2, 2, 2], [0, 0, 1, 1, 1, 1, 2, 2, 2], [0

3

In [226]:
BacktrackMultiple(backtrack, u,v,w)

7 5 8
7 4 8
6 3 7
5 3 7
5 2 7
5 2 6
4 1 5
3 1 5
2 1 5
2 1 4
2 1 3
2 1 2
-AT---ATC--CG-
--T-----C-C-GA
A-TGTA--CT--G-


In [235]:

a="ATCAACAACAGTTAGGATTATGTCCT"
b="GTACTGTGGAGGCTGCGTATGAGCC"
c="CGATCCCTGCCGGGACTAGGTATG"


In [236]:
score,backtrack=MultipleAlignment(a,b,c)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [233]:
score

11

In [237]:
BacktrackMultiple(backtrack, a,b,c)

26 25 24
25 25 24
24 25 24
23 25 24
22 25 24
22 24 24
22 23 24
22 22 24
22 21 24
21 20 23
20 19 22
19 18 21
18 18 21
18 17 21
18 16 21
18 15 21
18 14 21
17 13 20
16 13 20
16 12 20
15 11 19
14 10 18
13 9 17
12 9 17
12 8 17
12 7 17
12 6 17
12 5 17
11 4 16
10 4 16
9 4 16
8 4 16
7 4 16
6 4 16
5 3 15
4 3 15
3 2 14
2 2 14
2 2 13
2 2 12
2 2 11
2 2 10
2 2 9
2 2 8
2 2 7
2 2 6
2 2 5
2 2 4
1 1 3
----AT----------CAACAACAGT----TAGG-AT----TATG----TCCT
---G-T-----------A-C-----TGTGG-AGGC-TGCGT-ATGAGCC----
CGA--TCCCTGCCGGG-A-C-----T-----AGG--T-----ATG--------
